In [1]:
import sys
import gzip

import numpy as np
import tensorflow as tf

from binary_sayc import load_deals, DealData
from simulator import Simulator
from eval_bidding import eval_bidding

In [ ]:
sess = tf.InteractiveSession()
saver = tf.train.import_meta_graph('../model/sayc-500000.meta')
saver.restore(sess, '../model/sayc-500000')

graph = tf.get_default_graph()

seq_in = graph.get_tensor_by_name('seq_in:0')
seq_out = graph.get_tensor_by_name('seq_out:0')
keep_prob = graph.get_tensor_by_name('keep_prob:0')

out_bid_logit = graph.get_tensor_by_name('out_bid_logit:0')
out_bid_target = graph.get_tensor_by_name('out_bid_target:0')

In [3]:
def model(sess):
    def pred_fun(X):
        bids = sess.run(out_bid_logit, feed_dict={seq_in: X, keep_prob: 0.95})
        return sess.run(tf.nn.softmax(bids[-1]))
    return pred_fun

In [4]:
cost = tf.losses.softmax_cross_entropy(out_bid_target, out_bid_logit)

#train_step = tf.train.AdamOptimizer(0.001).minimize(cost)
train_step = graph.get_operation_by_name('softmax_w/Adam')

In [55]:
reader = load_deals(gzip.open('../deals_bidding_valid_0001.gz'))

In [ ]:
for i, (deal_str, auction_str, contracts) in enumerate(reader):
    if i % 10 == 0:
        sys.stdout.write('.')
    if i > 100:
        break
    deal_data = DealData.from_deal_auction_string(deal_str, auction_str)
    deal_data.reset_auction()
    init_auction = deal_data.auction
    
    sim = Simulator(deal_data, contracts, model(sess))
    
    best_score, best_auction = sim.best_auction(init_auction, 10)
    
    deal_data.auction = best_auction
    
    n_time_steps = (len(best_auction) // 4) + 1
    
    X, y = deal_data.get_binary(n_time_steps)
    
    sess.run(train_step, feed_dict={seq_in: X, seq_out: y, keep_prob: 0.8})

....

In [12]:
deal_str, auction_str, contracts = next(reader)

In [13]:
print(deal_str)
print(auction_str)

SQJT9 HKJ86 DAKJ7 CQ  S854 HQT7 DQ6 CAK872  S73 H42 D98432 CJ943  SAK62 HA953 DT5 CT65
S Both P P 1D 2C P 2D 2S 3C P P P


In [14]:
deal_data = DealData.from_deal_auction_string(deal_str, auction_str)
deal_data.reset_auction()

In [15]:
init_auction = deal_data.auction
init_auction

['PAD_START', 'PAD_START']

In [16]:
sim = Simulator(deal_data, contracts, model(sess))

In [42]:
sim.cache = {}

In [50]:
deal_data.auction

['PAD_START', 'PAD_START', 'PASS', 'PASS', '1D', '2C', 'PASS', '2D', '2S']

In [48]:
%%timeit
X, _ = deal_data.get_binary(3)

174 µs ± 2.09 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [54]:
%%timeit
model(sess)(X[1:2, 2:3])

671 ms ± 7.83 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [45]:
%%timeit 
sim.cache = {}; sim.simulate_bid(['PAD_START', 'PAD_START', 'PASS', 'PASS', '1D', '2C', 'PASS', '2D', '2S', ])

677 ms ± 10.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
sim.simulate_auction(init_auction)

['PAD_START',
 'PAD_START',
 'PASS',
 'PASS',
 '1D',
 '2C',
 'PASS',
 '2D',
 '2S',
 '3C',
 'PASS',
 'PASS',
 'PASS']

In [32]:
best_score, best_auction = sim.best_auction(init_auction, 10)
best_score, best_auction

(100.0,
 ['PAD_START',
  'PAD_START',
  'PASS',
  'PASS',
  'PASS',
  '1C',
  'PASS',
  '1D',
  'PASS',
  '2C',
  'PASS',
  '2D',
  'PASS',
  'PASS',
  'PASS'])

In [7]:
def apply_bidding(bid_model, fin, fout):
    reader = load_deals(fin)
    for i, (deal_str, auction_str, contracts) in enumerate(reader):
        if i % 10 == 0:
            sys.stdout.write('.')
        if i > 100:
            break
        deal_data = DealData.from_deal_auction_string(deal_str, auction_str)
        deal_data.reset_auction()
        init_auction = deal_data.auction
        sim = Simulator(deal_data, contracts, bid_model)
        auction = [('P' if bid == 'PASS' else bid) for bid in sim.simulate_auction(init_auction) if bid != 'PAD_START']
        fout.write((' ' + deal_str + '\n').encode('ascii'))
        fout.write((' ' + ' '.join(auction_str.split()[:2] + auction)).encode('ascii'))
        fout.write('\n'.encode('ascii'))
        for contract, (score1, score2) in contracts.items():
            fout.write(('{}\t{}\t{}\n'. format(contract, score1, score2)).encode('ascii'))

In [8]:
fout = gzip.open('../test1.gz', 'w')
apply_bidding(model(sess), gzip.open('../deals_bidding_valid_0001.gz'), fout)
fout.close()

...........

In [11]:
eval_bidding(gzip.open('../test1.gz'))

.

245.44554455445544